In [70]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi

In [51]:
tweets = pd.read_csv('data/train/tweets.queries.tsv', sep='\t', header=0, index_col=0)
tweets.head(5)

,tweet_content
106,Biden with Grand wizard Of KKK. So who again i...
274,"America should “build a wall of steel, a wall ..."
871,Thanks to White House Chief of Staff for this ...
876,"John Lewis said about my inauguration, ""It wil..."
138,Being a Christian is a insult to Muslims @Bern...


In [52]:
vclaims = pd.read_csv('data/verified_claims.docs.tsv', sep='\t', header=0, index_col=0)
vclaims.head(5)

,vclaim,title
0,122 detainees released from confinement at Gua...,Did 122 Prisoners Released from Guantanamo by ...
1,"A ""Trump and Obama by the Numbers"" meme recoun...",Does This Meme Accurately Show ‘Trump and Obam...
2,"A ""large-scale killing"" of white farmers is ta...",Is a ‘Large-Scale Killing’ of White Farmers Un...
3,"A ""law to separate families"" was enacted prior...",Was the ‘Law to Separate Families’ Passed in 1...
4,"A ""newly uncovered"" photograph reveals Alexand...",Does an Image Show Ocasio-Cortez Fake-Crying a...


In [58]:
qrels = pd.read_table('data/train/tweet-vclaim-pairs.qrels', 
                      names=['tweet_id', '0', 'vclaim_id', 'relevance'], 
                      index_col='tweet_id')
qrels.head(5)

,0,vclaim_id,relevance
tweet_id,,,
1,0,395,1
2,0,669,1
3,0,669,1
4,0,144,1
5,0,84,1


In [78]:
# function of preprocessing text, tokenization, stopwords removal, stemming
stopwords = stopwords.words('english')
url_pattern = r"(?:https?:\\/\\/(?:www\\.|(?!www))[^\\s\\.]+\\.[^\\s]{2,}|www\\.[^\\s]+\\.[^\\s]{2,})"
token_pattern = r"\b[A-Za-z][A-Za-z]+\b"

def preprocess_text(text, url_pattern = url_pattern, token_pattern=token_pattern, 
                    with_urlrm=True, with_stopwordsrm=True, stopwords=stopwords, with_stemming=False):
    # url removal 
    if with_urlrm == True:
        text = re.sub(url_pattern, "", text)
        
    # lower case 
    text_lower = text.lower()
    
    # tokenization 
    words = re.findall(token_pattern, text_lower)
    
    # stopwords removal
    if with_stopwordsrm == True:
        words = [word for word in words if word not in stopwords]
        
    # stemming 
    if with_stemming == True:
        ps = PorterStemmer() 
        words = [ps.stem(word) for word in words]
        
    text_processed = " ".join(words)
    
    return text_processed


In [79]:
tweets_prep = {}
for tweet_id in tweets.index:
    tweets_prep[tweet_id] = preprocess_text(tweets.loc[tweet_id, 'tweet_content'])

In [80]:
tweets.loc[106, 'tweet_content']

'Biden with Grand wizard Of KKK. So who again is playing you, lying to you, using you for the votes, Creators of the KKK, opposed civil right of blacks. Yup that’s the Democratic Party. pic.twitter.com/i5N4AN9Vb8 — UnclePhilly (@unclephilly) June 26, 2019'

In [81]:
tweets_prep[106]

'biden grand wizard kkk playing lying using votes creators kkk opposed civil right blacks yup democratic party pic twitter com unclephilly unclephilly june'